In [1]:
from pyspark.sql import Row
from pyspark.sql import SparkSession

warehouse_location = 'hdfs://hdfs:9000/hive/warehouse'

In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/22 08:43:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("CREATE TABLE IF NOT EXISTS iris (Id INT, SepalLengthCm DOUBLE, SepalWidthCm DOUBLE, PetalLengthCm DOUBLE, PetalWidthCm DOUBLE, Species STRING) USING hive")

21/12/22 08:43:24 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/12/22 08:43:24 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/12/22 08:43:27 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
21/12/22 08:43:28 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/12/22 08:43:28 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/12/22 08:43:28 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


DataFrame[]

In [4]:
spark.sql("LOAD DATA INPATH 'hdfs://hdfs:9000/user/Iris.csv' INTO TABLE iris")

21/12/22 08:43:30 ERROR KeyProviderCache: Could not find uri with key [dfs.encryption.key.provider.uri] to create a keyProvider !!
21/12/22 08:43:30 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]